# Knowledge base System
- 문서를 이용한 벡터DB 구성 실습

In [1]:
!pip install docx2txt

# 문서 로딩

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax.docx')
docu = loader.load()

# 문서 분할

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)
splitter

In [3]:
doc_list = loader.load_and_split(text_splitter=splitter)

In [4]:
len(doc_list)

183

# 임베딩

In [5]:
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

# 벡터 DB 적재 - chromaDB
- 인메모리 DB -> restart하면 날아감

In [ ]:
!pip install langchain-chroma

In [7]:
from langchain_chroma import Chroma
database = Chroma.from_documents(  # 새로 생성
    documents=doc_list,
    embedding=embedding,
    collection_name='chroma-tax',
    persist_directory='./chroma'
)

In [28]:
database = Chroma(   # 기존 저장본 불러오기
    collection_name='chroma-tax',
    persist_directory='./chroma',
    embedding_function=embedding
)

# Retrieval

In [8]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=5)

In [9]:
retrieved_docs

[Document(id='222a6d77-c35e-4b52-9f75-5b79061630fa', metadata={'source': './tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외

# augmentation

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini')

In [11]:
prompt = f"""
[Identity]
- 당신은 최고의 한국 소득세 전문가입니다.
- [context]내용만을 참조하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
"""

In [12]:
response = llm.invoke(prompt)

In [13]:
print(response.content)

연봉 5천만원인 직장인의 소득세는 아래의 세율을 적용하여 계산합니다.

1. **연봉 5천만원에 대한 과세표준**: 5천만원은 1,400만원 초과 ~ 5,000만원 이하 구간에 해당합니다.

2. **세율 적용**:
   - 기본 세액: 84만원 (1,400만원 초과 구간에 대한 기본 금액)
   - 추가 세액: 5,000만원 초과 금액 (즉, 5,000만원 - 1,400만원 = 3,600만원)에 대해 15% 적용.

계산식은 다음과 같습니다:
- 기본 세액: 84만원
- 추가 세액: 3,600만원 × 15% = 540만원

따라서 총 소득세는:
- 총 소득세 = 84만원 + 540만원 = 624만원

결과적으로, 연봉 5천만원인 직장인의 소득세는 **624만원**입니다.
